In [ ]:
import numpy as np
import pandas as pd
from warnings import filterwarnings as f_w
f_w('ignore')
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500
from itertools import product

In [ ]:
table = 'prev'
locator = pd.read_excel('locator.xlsx', index_col = 0)

In [ ]:
prev = pd.read_csv(locator.loc[table, 'initial'])

In [ ]:
for c in [co for co in prev.columns if 'DAYS' in co]:
    prev[c] = prev[c].replace({365243: np.nan})

In [ ]:
prev.loc[887093, 'CODE_REJECT_REASON'] = 'XNA' 
#это единственная запись, где отказано в кредите, но код причины ХРА, такой только у одобренных встречается

In [ ]:
prev.drop(1127152, inplace = True) 
#удалим единственную запись о кредите, у которой не указана сумма кредита

In [ ]:
for co in [c for c in prev.columns if prev[c].dtype == object]:
    prev[co] = prev[co].fillna('XNA')

In [ ]:
idx = prev[prev.CODE_REJECT_REASON == 'XNA'][prev.NAME_CONTRACT_STATUS == 'Approved'].index
prev.loc[idx, 'CODE_REJECT_REASON'] = 'XAP'

In [ ]:
prev = prev[prev.FLAG_LAST_APPL_PER_CONTRACT == 'Y'] #исключаем из анализа заявки, которые не являются последними по контракту, то есть их условия были изменены впоследствии или они вообще были ошибочными; непоследние заявки все отказные, кроме двух
prev.drop('FLAG_LAST_APPL_PER_CONTRACT', axis = 1, inplace = True)

In [ ]:
todrop = [
     'WEEKDAY_APPR_PROCESS_START', #в какой день недели началось одобрение
     'HOUR_APPR_PROCESS_START', #в какой час дня началось одобрение
     'NFLAG_LAST_APPL_IN_DAY', #последняя ли заявка за день, как установить другие заявки этого же клиента я не знаю
     'NAME_TYPE_SUITE', #c кем пришел в банк за кредитом
     'DAYS_FIRST_DRAWING', #этот показатель надо анализировать отдельно, вместе с данными о погашении кредитов
     'DAYS_FIRST_DUE', #этот показатель надо анализировать отдельно, вместе с данными о погашении кредитов
     'DAYS_LAST_DUE_1ST_VERSION', #этот показатель надо анализировать отдельно, вместе с данными о погашении кредитов
     'DAYS_LAST_DUE', #этот показатель надо анализировать отдельно, вместе с данными о погашении кредитов
     'DAYS_TERMINATION', #этот показатель надо анализировать отдельно, вместе с данными о погашении кредитов
     'NFLAG_INSURED_ON_APPROVAL', #застрахован ли кредит
     'SELLERPLACE_AREA', #площадь офиса, где был оформлен кредит
     'NAME_SELLER_INDUSTRY', #отрасль компании, где был оформлен кредит
     'NAME_PORTFOLIO', #в основном информация дублируется в PRODUCT_COMBINATION
    ]

prev = prev.drop(columns = todrop)
#пока удаляем эти колонки, как бы гипотеза о том, что они не влияют существенно на 
#вероятность возврата кредита или должны быть проанализированы отдельно

In [ ]:
prev['CREDIT_APPL'] = prev.eval('AMT_CREDIT / AMT_APPLICATION')
prev['APPL_PRICE'] = prev.eval('AMT_CREDIT / AMT_GOODS_PRICE')
prev['DAYS_DECISION_DIFF'] = prev.groupby('SK_ID_CURR')['DAYS_DECISION'].diff()

In [ ]:
prev = prev[(prev.NAME_CONTRACT_STATUS == 'Approved') | (prev.NAME_CONTRACT_STATUS == 'Refused')]
prev = prev.sort_values(by = ['SK_ID_CURR', 'DAYS_DECISION'])
approved = prev[prev.NAME_CONTRACT_STATUS == 'Approved'].drop(columns = 'NAME_CONTRACT_STATUS')
refused = prev[prev.NAME_CONTRACT_STATUS == 'Refused'].drop(columns = 'NAME_CONTRACT_STATUS')

In [ ]:
cat_agg = {'NAME_CONTRACT_STATUS': ['last', lambda x: x.mode()[0]], #статус по последней заявке и самый частый статус
           'NAME_CASH_LOAN_PURPOSE': ['last', lambda x: x.mode()[0]],  #цель последней заявки
           'CODE_REJECT_REASON': ['last', lambda x: x.mode()[0]], #причина отказа по последней заявке
           'NAME_GOODS_CATEGORY': ['last', lambda x: x.mode()[0]], #категория товаров по последней заявке
           'NAME_CLIENT_TYPE': ['first', 'last'], #тип клиента по первой и последней заявкам
           'NAME_YIELD_GROUP': ['last', lambda x: x.mode()[0]],
           'PRODUCT_COMBINATION': ['last', lambda x: x.mode()[0]],
           'CHANNEL_TYPE': ['last', lambda x: x.mode()[0]],
          }  

In [ ]:
num_agg = {'SK_ID_PREV': 'count', #сколько заявок подавал клиент
           'NAME_CONTRACT_TYPE' : 'nunique', #на сколько продуктов банка претендовал клиент
           'NAME_CASH_LOAN_PURPOSE': 'nunique',
           'AMT_ANNUITY': ['min', 'max', 'mean', 'std'],
           'AMT_APPLICATION': ['min', 'max', 'mean', 'sum', 'last'],
           'AMT_CREDIT': ['min', 'max', 'mean', 'sum', 'last'],
           'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
           'CREDIT_APPL': ['min', 'max', 'mean', 'std'],
           'APPL_PRICE': ['min', 'max', 'mean', 'std'],
           'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'std'],
           'RATE_DOWN_PAYMENT': ['min', 'max', 'mean', 'std'],
           'RATE_INTEREST_PRIMARY': ['mean', 'std'],
           'RATE_INTEREST_PRIVILEGED': 'mean',
           'DAYS_DECISION': ['min', 'max'], #как давно было принято решение по первой заявке, поданной клиентом; когда была рассмотрена последняя заявка, поданная клиентом
           'DAYS_DECISION_DIFF': ['min', 'max', 'mean', 'std', 'last'],
           'CNT_PAYMENT': ['min', 'max', 'mean', 'std', 'sum', 'last'],
          }  

In [ ]:
def get_prelast(df, column, nth, name):
    return df.groupby('SK_ID_CURR').nth(nth)[column].rename(name)
#функция чтобы получить предпоследнюю и предпредпоследнюю запись в какой-то колонке, сгруппированной по SK_ID_CURR

In [ ]:
def get_reject_reasons_means(df = prev):
    dummies = pd.get_dummies(df[['SK_ID_CURR', 'CODE_REJECT_REASON']])
    try:
        dummies = dummies.drop(columns = ['CODE_REJECT_REASON_CLIENT', 'CODE_REJECT_REASON_XAP']) 
    except:
        pass
    grouped = dummies.groupby('SK_ID_CURR').agg('mean')
    grouped.columns = pd.Index([(c, '') for c in grouped.columns.tolist()])
    return grouped

In [ ]:
prev_agg = prev.groupby('SK_ID_CURR').agg({**cat_agg, **num_agg})
prev_agg = prev_agg.join(get_reject_reasons_means())
for col, nth in product(cat_agg.keys(), [-2, -3]):
    name = ('{}_LAST'.format(col), str(nth))
    prev_agg = prev_agg.join(get_prelast(prev, col, nth, name))
prev_agg.columns = pd.Index(['PREV_' + c[0] + "_" + c[1].upper().replace('<LAMBDA>', 'MODE') 
                             for c in prev_agg.columns.tolist()])

In [ ]:
cat_agg.pop('NAME_CONTRACT_STATUS')
ref_agg = refused.groupby('SK_ID_CURR').agg({**cat_agg, **num_agg})
for col, nth in product(cat_agg.keys(), [-2, -3]):
    name = ('{}_LAST'.format(col), str(nth))
    ref_agg = ref_agg.join(get_prelast(refused, col, nth, name))
ref_agg.columns = pd.Index(['PREV_REFUSED_' + c[0] + "_" + c[1].upper().replace('<LAMBDA>', 'MODE') 
                            for c in ref_agg.columns.tolist()])

In [ ]:
cat_agg.pop('CODE_REJECT_REASON')
app_agg = approved.groupby('SK_ID_CURR').agg({**cat_agg, **num_agg})
for col, nth in product(cat_agg.keys(), [-2, -3]):
    name = ('{}_LAST'.format(col), str(nth))
    app_agg = app_agg.join(get_prelast(approved, col, nth, name))
app_agg.columns = pd.Index(['PREV_APPROVED_' + c[0] + "_" + c[1].upper().replace('<LAMBDA>', 'MODE') 
                            for c in app_agg.columns.tolist()])

In [ ]:
prev_agg = prev_agg.join(app_agg)
prev_agg = prev_agg.join(ref_agg)

In [ ]:
todrop = prev_agg.columns[prev_agg.isnull().sum().eq(prev_agg.shape[0])]
prev_agg = prev_agg.drop(columns = todrop)

In [ ]:
prev_agg = prev_agg.replace({np.inf:np.nan,
                             -np.inf:np.nan})

In [ ]:
prev_agg.to_csv(locator.loc[table, 'feat_eng'])